In [3]:
!cd

D:\Ansys Simulations\Project\2D


# SCALING NOTEBOOK
This notebook is used to develop the functions needed to script the scaling of the data

In [10]:
## imports
from pathlib import Path
import pandas as pd

In [9]:
## Create a function to get the data
def get_sample(samples_folder_path, sample_number):
    ## returns the input and output dataframe for the sample specified
    
    input_path = Path()

data_folder_path =  Path('D:/Ansys Simulations/Project/2D/data') 
print(data_folder_path)
get_sample(data_folder_path, 26)

D:\Ansys Simulations\Project\2D\data
